In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Category:Subdivisions_of_Rome').text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()

,Neighborhood
0,Administrative subdivision of Rome
1,Colle Salario
2,List of shopping areas and markets in Rome
3,14 regions of Augustan Rome
4,14 regions of Medieval Rome


In [4]:
df.shape

(34, 1)

In [7]:
address = 'rome , italy'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of rome are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of rome are 41.8933203, 12.4829321.


### 3. Get the geographical coordinates


In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, rome , italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [10]:
coords

[[6.213510000000042, 1.1898700000000417],
 [41.91331000000008, 12.502170000000035],
 [41.90322000000003, 12.495650000000069],
 [41.923008996117446, 12.609025115981924],
 [41.923008996117446, 12.609025115981924],
 [41.78340000000003, 12.365110000000072],
 [41.9228899233944, 12.438710080500838],
 [41.91589000000005, 12.36955000000006],
 [41.84964000000008, 12.574010000000044],
 [41.83449007328752, 12.472889976052295],
 [41.92999000000003, 12.464510000000075],
 [41.943700051070486, 12.474690043857139],
 [41.862430000000074, 12.49007000000006],
 [41.73499000000004, 12.362240000000043],
 [42.009870000000035, 12.378480000000025],
 [41.99281000000008, 12.489860000000022],
 [41.90322000000003, 12.495650000000069],
 [41.83674000000008, 12.43011000000007],
 [41.879000000000076, 12.350150000000042],
 [41.75968000000006, 12.300710000000038],
 [41.76027000000005, 12.301680000000033],
 [42.124450000000024, 12.289570000000026],
 [41.82144000000005, 12.350910000000056],
 [42.003720000000044, 12.481950

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
df

,Neighborhood,Latitude,Longitude
0,Administrative subdivision of Rome,6.213510,1.189870
1,Colle Salario,41.913310,12.502170
2,List of shopping areas and markets in Rome,41.903220,12.495650
3,14 regions of Augustan Rome,41.923009,12.609025
4,14 regions of Medieval Rome,41.923009,12.609025
5,Acilia,41.783400,12.365110
6,Balduina,41.922890,12.438710
7,Casalotti,41.915890,12.369550
8,Cinecittà,41.849640,12.574010
9,Columbus (Rome),41.834490,12.472890


In [14]:
df.shape

(34, 3)

### 4. Create a map of rome with neighborhoods superimposed on top


In [15]:
# create map of rome using latitude and longitude values
map_da = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'],df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_da)  
    
map_da

### 5. Use the Foursquare API to explore the neighborhoods


In [16]:
# Define Foursquare Credentials and Version
LIMIT = 100

CLIENT_ID = 'AJMPDDKDSO2EBLSGK4OCMSQPSAT1MSVVPFQEAWWG3J0RO2UC' # your Foursquare ID
CLIENT_SECRET = 'JV3VTEOZU21SVVOHQEVE4MANEBZF3I430BXURCS4YW0V3XIP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AJMPDDKDSO2EBLSGK4OCMSQPSAT1MSVVPFQEAWWG3J0RO2UC
CLIENT_SECRET:JV3VTEOZU21SVVOHQEVE4MANEBZF3I430BXURCS4YW0V3XIP


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [17]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [18]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1624, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Administrative subdivision of Rome,6.21351,1.18987,Au Bon Pain,6.206687,1.189566,Coffee Shop
1,Administrative subdivision of Rome,6.21351,1.18987,Le Hangar,6.205295,1.196624,Restaurant
2,Administrative subdivision of Rome,6.21351,1.18987,Marché de Cacaveli,6.205212,1.196681,Market
3,Administrative subdivision of Rome,6.21351,1.18987,Kastas Fast Food,6.227165,1.201411,Fast Food Restaurant
4,Administrative subdivision of Rome,6.21351,1.18987,LA CROISSANTERIE,6.195600,1.189310,Sandwich Place


In [20]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
14 regions of Augustan Rome,23,23,23,23,23,23
14 regions of Medieval Rome,23,23,23,23,23,23
Acilia,34,34,34,34,34,34
Administrative subdivision of Rome,5,5,5,5,5,5
Balduina,100,100,100,100,100,100
Casalotti,15,15,15,15,15,15
Cinecittà,78,78,78,78,78,78
Colle Salario,100,100,100,100,100,100
Columbus (Rome),100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [21]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 177 uniques categories.


In [22]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:20]

array(['Coffee Shop', 'Restaurant', 'Market', 'Fast Food Restaurant',
       'Sandwich Place', 'Dessert Shop', 'Ice Cream Shop', 'Juice Bar',
       'Monument / Landmark', 'Italian Restaurant',
       'Vegetarian / Vegan Restaurant', 'Chinese Restaurant',
       'Steakhouse', 'Noodle House', 'Art Museum', 'Park', 'Pizza Place',
       'Hotel', 'Greek Restaurant', 'Plaza'], dtype=object)

### 6. Analyze Each Neighborhood

In [23]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1624, 178)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Argentinian Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Camera Store,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Library,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Rest Area,Restaurant,Road,Rock Club,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Soccer Stadium,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,Administrative subdivision of Rome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Administrative subdivision of Rome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Administrative subdivision of Rome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Administrative subdivision of Rome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Administrative subdivision of Rome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [25]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(34, 178)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Argentinian Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Café,Camera Store,Campground,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Lake,Library,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Resort,Rest Area,Restaurant,Road,Rock Club,Roman Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Soccer Stadium,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,14 regions of Augustan Rome,0.00,0.000000,0.043478,0.00,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.130435,0.00,0.000000,0.000000,0.043478,0.173913,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.086957,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.043478,0.00,0.00000,0.00,0.00,0.000000,0.0000,0.000000,0.043478,0.00000,0.043478,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.043478,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.043478,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00000
1,14 regions of Medieval Rome,0.00,0.000000,0.043478,0.00,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.00,0.0

In [26]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

6

In [27]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [28]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,14 regions of Augustan Rome,0.043478
1,14 regions of Medieval Rome,0.043478
2,Acilia,0.000000
3,Administrative subdivision of Rome,0.000000
4,Balduina,0.000000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in damascus into 3 clusters.

In [29]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [31]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,14 regions of Augustan Rome,0.043478,0
1,14 regions of Medieval Rome,0.043478,0
2,Acilia,0.000000,1
3,Administrative subdivision of Rome,0.000000,1
4,Balduina,0.000000,1


In [32]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(34, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,14 regions of Augustan Rome,0.043478,0,41.923009,12.609025
1,14 regions of Medieval Rome,0.043478,0,41.923009,12.609025
2,Acilia,0.000000,1,41.783400,12.365110
3,Administrative subdivision of Rome,0.000000,1,6.213510,1.189870
4,Balduina,0.000000,1,41.922890,12.438710


In [33]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(34, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,14 regions of Augustan Rome,0.043478,0,41.923009,12.609025
1,14 regions of Medieval Rome,0.043478,0,41.923009,12.609025
31,Torrenova (Rome),0.050000,0,41.859240,12.622480
28,Spinaceto,0.045455,0,41.780450,12.439300
19,Ostia (Rome),0.000000,1,41.759680,12.300710
20,Ostia Antica (district),0.000000,1,41.760270,12.301680
21,Polline Martignano,0.000000,1,42.124450,12.289570
22,Ponte Galeria,0.000000,1,41.821440,12.350910
23,Prima Porta,0.000000,1,42.003720,12.481950
25,Rebibbia,0.000000,1,41.927110,12.574670


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,14 regions of Augustan Rome,0.043478,0,41.923009,12.609025
1,14 regions of Medieval Rome,0.043478,0,41.923009,12.609025
31,Torrenova (Rome),0.050000,0,41.859240,12.622480
28,Spinaceto,0.045455,0,41.780450,12.439300


In [36]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
19,Ostia (Rome),0.000000,1,41.759680,12.300710
20,Ostia Antica (district),0.000000,1,41.760270,12.301680
21,Polline Martignano,0.000000,1,42.124450,12.289570
22,Ponte Galeria,0.000000,1,41.821440,12.350910
23,Prima Porta,0.000000,1,42.003720,12.481950
25,Rebibbia,0.000000,1,41.927110,12.574670
18,Massimina,0.000000,1,41.879000,12.350150
26,San Lorenzo (Rome),0.000000,1,41.896680,12.514770
27,Settecamini,0.000000,1,41.937970,12.620700
29,Tor Cervara,0.000000,1,41.916599,12.589191


In [37]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,Infernetto,0.0625,2,41.73499,12.36224
